# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [71]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import base64
import json

def get_key(keyfile ='config.py'):
    with open(keyfile,'rb') as inf:
        k = base64.b64decode(inf.read())
        inf.close()
    return k.decode('utf8')

gkey = get_key()

gmaps.configure(gkey)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [72]:
city_data = pd.read_csv("Resources/city_data.csv")
city_data.head()

,Unnamed: 0,City,Country,Date,Latitude,Longitude,Humidity,Max Temp,Wind Speed,Cloudiness
0,0,Galesong,ID,1584908956,-5.32,119.37,100,75.20,2.24,20
1,1,Mar del Plata,AR,1584908899,-38.00,-57.56,42,82.40,6.93,54
2,2,Grimshaw,CA,1584908956,56.18,-117.60,55,33.80,5.82,90
3,3,Nome,US,1584908956,64.50,-165.41,85,21.20,4.70,90
4,4,Albany,US,1584908956,42.60,-73.97,25,42.01,6.93,20


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [7]:
!jupyter nbextension enable --py gmaps

Enabling notebook extension jupyter-gmaps/extension...
      - Validating: OK


In [73]:
locations = city_data[["Latitude", "Longitude"]]
humidity = city_data["Humidity"]

In [74]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights = humidity, 
                                 dissipating = False, max_intensity = 100, point_radius = 5)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [75]:
ideal_spots = city_data

ideal_spots = ideal_spots.loc[(ideal_spots["Max Temp"] < 80) & (ideal_spots["Max Temp"] > 70)]

ideal_spots = ideal_spots.loc[ideal_spots["Wind Speed"] < 10]

ideal_spots = ideal_spots.loc[ideal_spots["Cloudiness"] == 0]

ideal_spots.head()

,Unnamed: 0,City,Country,Date,Latitude,Longitude,Humidity,Max Temp,Wind Speed,Cloudiness
7,7,Carnarvon,AU,1584908957,-24.87,113.63,74,78.80,8.05,0
13,13,Hong Kong,HK,1584908957,22.29,114.16,88,75.00,4.70,0
63,63,Chui,UY,1584908961,-33.70,-53.46,52,75.07,9.10,0
103,103,Castro,BR,1584908964,-24.79,-50.01,61,70.03,4.05,0
131,131,Lolodorf,CM,1584908967,3.23,10.73,97,74.17,1.95,0


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [76]:
hotel_df = ideal_spots
hotel_df["Hotel Name"] = ""

In [109]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {
    "types" : "lodging",
    "radius" : 500,
    "key" : gkey
}

for index, row in hotel_df.iterrows():
    city = row["City"]
    lat = row["Latitude"]
    lon = row["Longitude"]
    
    params["location"] = f"{lat}, {lon}"
    
    print(f"Finding hotel within 5000m in {city}")
    print(f"Retrieving results for Index {index}: {lat}, {lon}")
    response = requests.get(base_url, params = params).json()
    print(json.dumps(response, indent = 4, sort_keys = True))
    
    results = response["results"]
    
    try: 
        hotel_df.loc[index, "Hotel Name"] = results[0]["name"]
        print("-----------------------------------------------------------------------------")     
    
    except(KeyError, IndexError):
        print("Missing field/result ... skipping.")
        print("-----------------------------------------------------------------------------")
    

Finding hotel within 5000m in Carnarvon
Retrieving results for Index 7: -24.87, 113.63
{
    "html_attributions": [],
    "results": [],
    "status": "ZERO_RESULTS"
}
Missing field/result ... skipping.
-----------------------------------------------------------------------------
Finding hotel within 5000m in Hong Kong
Retrieving results for Index 13: 22.29, 114.16
{
    "html_attributions": [],
    "results": [],
    "status": "ZERO_RESULTS"
}
Missing field/result ... skipping.
-----------------------------------------------------------------------------
Finding hotel within 5000m in Chui
Retrieving results for Index 63: -33.7, -53.46
{
    "html_attributions": [],
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -33.6997931,
                    "lng": -53.46408880000001
                },
                "viewport": {
                    "northeast": {
                        "lat": -33.6984904697085,
                      

{
    "html_attributions": [],
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 3.2332704,
                    "lng": 10.7281001
                },
                "viewport": {
                    "northeast": {
                        "lat": 3.235503599999999,
                        "lng": 10.7292491802915
                    },
                    "southwest": {
                        "lat": 3.232526,
                        "lng": 10.7265512197085
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "id": "dd5b8c54656bdea2b9ee0953641f9d73ca29fa47",
            "name": "Auberge La Decouverte",
            "photos": [
                {
                    "height": 1536,
                    "html_attributions": [
                        "<a href=\"https://maps.google.com/maps/contrib/110822375882786649610\">Bernard Bogo</

{
    "html_attributions": [],
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 24.4324673,
                    "lng": 80.0340849
                },
                "viewport": {
                    "northeast": {
                        "lat": 24.4338162802915,
                        "lng": 80.03543388029149
                    },
                    "southwest": {
                        "lat": 24.4311183197085,
                        "lng": 80.03273591970849
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "id": "6359ab65ce14617470ff227e3801845c4abcfab2",
            "name": "Kalpana Inn amanganj",
            "photos": [
                {
                    "height": 4096,
                    "html_attributions": [
                        "<a href=\"https://maps.google.com/maps/contrib/101443969975305247299\">Saran

In [107]:
hotel_df

,Unnamed: 0,City,Country,Date,Latitude,Longitude,Humidity,Max Temp,Wind Speed,Cloudiness,Hotel Name
7,7,Carnarvon,AU,1584908957,-24.87,113.63,74,78.80,8.05,0,
13,13,Hong Kong,HK,1584908957,22.29,114.16,88,75.00,4.70,0,
63,63,Chui,UY,1584908961,-33.70,-53.46,52,75.07,9.10,0,Gaboto
103,103,Castro,BR,1584908964,-24.79,-50.01,61,70.03,4.05,0,Hotel Buganville Express
131,131,Lolodorf,CM,1584908967,3.23,10.73,97,74.17,1.95,0,Auberge La Decouverte
135,135,Río Blanco,MX,1584908967,18.83,-97.15,35,77.23,6.15,0,Departamentos Dorantds
202,202,Āsosa,ET,1584908973,10.07,34.53,15,75.25,5.88,0,
259,259,Allāpalli,IN,1584908978,19.42,80.07,25,76.51,2.53,0,
267,267,Jiazi,CN,1584908978,22.88,116.07,88,70.93,2.89,0,
277,277,Harsūd,IN,1584908979,22.10,76.73,23,78.13,5.01,0,


In [115]:
# NOTE: Do not change any of the code in this cell
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [117]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.symbol_layer(
    locations, fill_color = 'rgba(0, 150, 0, 0.4)',
    stroke_color = 'rgba(0, 0, 150, 0.4)', scale = 5,
    info_box_content = hotel_info
)

# Display Map
fig = gmaps.figure()
fig.add_layer(hotel_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [118]:
fig = gmaps.figure()

fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)

fig

Figure(layout=FigureLayout(height='420px'))